## Utilizando o Apache Spark e demais ferramentas correlatas, implemente os seguintes passos:

1. Selecione um estado brasileiro e dez de suas cidades. Crie um CSV paraarmazenar as cidades, com id (nome da cidade), latitude, longitude e população. Crie outro CSV para armazenar a distância entre essas cidades, com src, dst e relationship como campos (adicione pelo menos 30 registros nesse arquivo).

2. Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

3. Utilizando o método bfs (Breadth First Search), execute 5 filtragens a sua escolha.

4. Execute 2 consultas utilizando o método find.

5. Execute 2 consultas utilizando o método filterVertices.

6. Implemente uma rotina que, recebendo como entrada um objeto GraphFrame, percorra todos os vértices do grafo com o algoritmo da busca em profundidade.

In [1]:
from graphframes import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf

### 1&2  Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

In [2]:
conf = SparkConf().setAppName('myapp').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)
nodes = spark.read.csv('data/transport-nodes.csv', header=True)
rels = spark.read.csv('data/transport-relationships.csv', header=True)
g = GraphFrame(nodes, rels)

In [3]:
g.vertices.show()

+----------------+---------+----------+----------+
|              id| latitude| longitude|population|
+----------------+---------+----------+----------+
|       Araripina|52.379189|  4.899431|    821752|
|         Caruaru|52.092876|   5.10448|    334176|
|        Igarassu|52.078663|  4.288788|    514861|
|       Instancia| 53.61239|  -0.22219|      9642|
|         Carpina| 53.52285|  -1.13116|    302400|
|   Serra Talhada|  51.9775|   4.13333|      9382|
|      Arco Verde| 51.96375|    1.3511|     23689|
|        Ouricuri| 52.05917|   1.15545|    133384|
|         Surubim| 51.88921|   0.90421|    104390|
|       Petrolina|51.509865| -0.118092|   8787892|
|       Salgueiro|  51.9225|   4.47917|    623652|
|             Exu| 52.01667|   4.70833|     70939|
|          Cupira|-8.616944|    -35.95|    24 107|
|           Orobó|   -7.745|-35.601944|    23 884|
|     Rio Formoso|-8.663889|-35.158889|    23 535|
|         Amaraji|-8.382778|-35.452778|    22 829|
|      Feira Nova|-7.950833|-35

In [15]:
g.edges.show(30)

+----------------+----------------+------------+----+
|             src|             dst|relationship|cost|
+----------------+----------------+------------+----+
|       Araripina|         Caruaru|       EROAD|  46|
|       Araripina|        Igarassu|       EROAD|  59|
|        Igarassu|       Salgueiro|       EROAD|  26|
|       Araripina|       Instancia|       EROAD| 369|
|       Instancia|         Carpina|       EROAD|  74|
|         Carpina|       Petrolina|       EROAD| 277|
|   Serra Talhada|        Igarassu|       EROAD|  27|
|      Arco Verde|   Serra Talhada|       EROAD| 207|
|        Ouricuri|      Arco Verde|       EROAD|  22|
|         Surubim|        Ouricuri|       EROAD|  32|
|       Petrolina|         Surubim|       EROAD| 106|
|             Exu|       Salgueiro|       EROAD|  25|
|             Exu|         Caruaru|       EROAD|  35|
|        Igarassu|             Exu|       EROAD|  32|
|   Serra Talhada|       Salgueiro|       EROAD|  33|
|          Cupira|          

### 3. Utilizando o método bfs (Breadth First Search), execute 5 filtragens a sua escolha.

In [5]:
paths = g.bfs("id = 'Araripina'", "id = 'Salgueiro'")
paths.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Araripina, 52.37...|[Araripina, Igara...|[Igarassu, 52.078...|[Igarassu, Salgue...|[Salgueiro, 51.92...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [6]:
paths = g.bfs("id = 'Araripina'", "id = 'Salgueiro'", 
              edgeFilter = "relationship != 'EROAD'")
paths.show()

+---+--------+---------+----------+
| id|latitude|longitude|population|
+---+--------+---------+----------+
+---+--------+---------+----------+



In [16]:
paths = g.bfs("id = 'Igarassu'", "id = 'Lagoa de Itaenga'")
paths.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  v5|                  e5|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Igarassu, 52.078...|[Igarassu, Exu, E...|[Exu, 52.01667, 4...|[Exu, Amaraji, ER...|[Amaraji, -8.3827...|[Amaraji, Ouricur...|[Ouricuri, 52.059...|[Ouricuri, Arco V...|[Arco 

In [8]:
paths = g.bfs("id = 'Exu'", "id = 'Arco Verde'")
paths.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Exu, 52.01667, 4...|[Exu, Amaraji, ER...|[Amaraji, -8.3827...|[Amaraji, Ouricur...|[Ouricuri, 52.059...|[Ouricuri, Arco V...|[Arco Verde, 51.9...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [9]:
paths = g.bfs("id = 'Petrolina'", "id = 'Caruaru'")
paths.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  v5|                  e5|                  v6|                  e6|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Petrolina, 51.50...|[Petrolina, Surub...|[Surub

### 4. Execute 2 consultas utilizando o método find.

In [10]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(c)")
motifs.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                   e|                   b|                  e2|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Araripina, 52.37...|[Araripina, Igara...|[Igarassu, 52.078...|[Igarassu, Exu, E...|[Exu, 52.01667, 4...|
|[Araripina, 52.37...|[Araripina, Igara...|[Igarassu, 52.078...|[Igarassu, Salgue...|[Salgueiro, 51.92...|
|[Araripina, 52.37...|[Araripina, Insta...|[Instancia, 53.61...|[Instancia, Carpi...|[Carpina, 53.5228...|
|[Instancia, 53.61...|[Instancia, Carpi...|[Carpina, 53.5228...|[Carpina, Petroli...|[Petrolina, 51.50...|
|[Carpina, 53.5228...|[Carpina, Petroli...|[Petrolina, 51.50...|[Petrolina, Surub...|[Surubim, 51.8892...|
|[Serra Talhada, 5...|[Serra Talhada, I...|[Igarassu, 52.078...|[Igarassu, Exu, E...|[Exu, 52.01667, 4...|
|[Serra Talhada, 5...|[Serra Talhada,

In [17]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(c); (c)-[e3]->(a)")
motifs.show()

+---+---+---+---+---+---+
|  a|  e|  b| e2|  c| e3|
+---+---+---+---+---+---+
+---+---+---+---+---+---+



### 5. Execute 2 consultas utilizando o método filterVertices.

In [12]:
g1 = g.filterVertices("population < 10000")
g1.vertices.show()

+-------------+--------+---------+----------+
|           id|latitude|longitude|population|
+-------------+--------+---------+----------+
|    Instancia|53.61239| -0.22219|      9642|
|Serra Talhada| 51.9775|  4.13333|      9382|
+-------------+--------+---------+----------+



In [13]:
g1 = g.filterVertices("latitude > 52.1")
g1.vertices.show()

+---------+---------+---------+----------+
|       id| latitude|longitude|population|
+---------+---------+---------+----------+
|Araripina|52.379189| 4.899431|    821752|
|Instancia| 53.61239| -0.22219|      9642|
|  Carpina| 53.52285| -1.13116|    302400|
+---------+---------+---------+----------+



### 6. Implemente uma rotina que, recebendo como entrada um objeto GraphFrame, percorra todos os vértices do grafo com o algoritmo da busca em profundidade.